In [206]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googletrans import Translator
import re
from collections import Counter

In [2]:
def get_quote_by_search(keyword,loop_pagination=50,to_csv=False,display_print=False,enable_translate=False,translate_src='en',translate_dest='id'):
    quotes_origin=[] #define the temporary bucket
    quotes_translated = [] #define the temporary bucket
    iteration_count=1 #not mandatory, for printing purposes to count the content scraped

    for i in range(loop_pagination):
        current_loop_url = "https://www.brainyquote.com/search_results?q={}&pg={}".format(keyword,i+1) #the search url
        page=requests.get(current_loop_url) #fetching html
        if display_print:
            print("Scrapping from {}".format(current_loop_url))
        soup = BeautifulSoup(page.content,'html.parser') #parsing html to beautifulsoup
        mydivs = soup.find_all("div", {"class": "grid-item qb clearfix bqQt"}) #find the html loops to scrape
        for items in mydivs: #loops
            title_element = items.find("div",{"style":"display: flex;justify-content: space-between"}) #get the title element
            author_element = items.find("a",{"title":"view author"}) #get the author element
            output = title_element.text.replace('\n','')+" - "+author_element.text #concatenating title and author
            output_translated = (translator.translate(output,src='en',dest='id')).text
            quotes_origin.append(output) #adding the concated title and outhor to temporary bucket
            quotes_translated.append(output_translated)
            if display_print:
                print("-----{} content retrieved".format(iteration_count)) #counting the contents
            iteration_count+=1 #iteration count

    output_df = pd.DataFrame({ #building the dataframe
            'quotes_origin':quotes_origin
            ,'quotes_translated': quotes_translated
        }).drop_duplicates()
        
    if  to_csv:
        csv_filename="Brainyquote search by {}.csv".format(keyword)
        output_df.drop_duplicates().to_csv(csv_filename,index=False,sep='|')
        print(csv_filename+" created successfully.")
    return output_df #returning the dataframe

In [3]:
translator = Translator()
# output = get_quote_by_search("business",to_csv=True)

In [5]:
page = requests.get('https://www.brainyquote.com/topics')
soup = BeautifulSoup(page.content,'html.parser')

In [296]:
list_topic,list_quote_text,list_quote_author_text,len_debug_soup=[],[],[],[]
page = requests.get('https://www.brainyquote.com/topics')
soup = BeautifulSoup(page.content,'html.parser')
topic_url = soup.find_all("a", {"class": "topicIndexChicklet bq_on_link_cl"}) #find the html loops to scrape
for items in topic_url:
    for iteration in range(1):
        url_to_scrape='https://brainyquotes.com'+items['href']+'_'+str(iteration+1) #build the url
#         print("Scraping from '{}'".format(url_to_scrape))
        topic = items.get_text() #get_topic_string
        page = requests.get(url_to_scrape) #go to url
        soup = BeautifulSoup(page.content,'html.parser')
        soup_quote = soup.find_all("div", {"class": "grid-item qb clearfix bqQt"}) #find the html loops to scrape
        for items in soup_quote:
            try:
#                 quote_text_author = items.get_text().replace('\n','').replace('\'','')
#                 quote_author = re.search('[\.\!\?\']\w.*$',quote_text_author)

#                 if len(items) == 7:
#                     quote_author_text = re.search('[\.\!\?\']\w.*$',items.get_text().replace('\n','').replace('\'','')).group(0)
#                 elif len(items) == 5:
#                     quote_author_text = re.search('[\.\!\?\']\w.*$',items.get_text().replace('\n','').replace('\'','')).group(0)
                quote_author_text = re.search('[\.\!\?\']\w.*$',items.get_text().replace('\n','').replace('\'','')).group(0)
                quote_text = quote_text_author.replace(quote_author_text,'')+'.'
                len_debug_soup.append(len(items))
                list_topic.append(topic.replace('\n',''))
                list_quote_text.append(quote_text)
                list_quote_author_text.append(quote_author_text.replace('.',''))
            except:
                print("'{}'-'{}".format(items.get_text().replace('\n',''),url_to_scrape))
                continue
        break
print("DONE")

DONE


In [297]:
list_quote_author_text

['George Sand',
 'Mark Twain',
 'Henry Ford',
 'Stanislaw Jerzy Lec',
 'Victor Hugo',
 'George Burns',
 'Kurt Vonnegut',
 'George Bernard Shaw',
 'Sophia Loren',
 'Arthur Schopenhauer',
 'Lucille Ball',
 'Andrew Carnegie',
 'Confucius',
 'Tom Stoppard',
 'Betty Friedan',
 'James Broughton',
 'Sam Ewing',
 'Bette Davis',
 'Samuel Ullman',
 'Henry David Thoreau',
 'Pope John XXIII',
 'John Adams',
 'Robert Frost',
 'Jules Renard',
 'Anais Nin',
 'Eleanor Roosevelt',
 'Rabindranath Tagore',
 'Theodore Roosevelt',
 'Marvin Davis',
 'Golda Meir',
 'Dylan Thomas',
 'Dr Seuss',
 'Helen Hayes',
 'C S Lewis',
 'Gertrude Stein',
 'Agatha Christie',
 'Sophocles',
 'Coco Chanel',
 'Emily Dickinson',
 'Josh Billings',
 'Oscar Wilde',
 '!Pierre Corneille',
 'Bob Hope',
 'Charles de Gaulle',
 'Jonathan Swift',
 'Casey Stengel',
 'Albert Camus',
 'Thomas Browne',
 'Pope Paul VI',
 'B F Skinner',
 'Voltaire',
 'T S Eliot',
 'Woody Harrelson',
 'John Singer Sargent',
 'Muriel Spark',
 'Amos Bronson Alco

In [183]:
debug_url = 'https://www.brainyquote.com/topics/age-quotes_1'
debug_page = requests.get(debug_url) #go to url
soup = BeautifulSoup(debug_page.content,'html.parser')
debug_soup_quote = soup.find_all("div", {"class": "grid-item qb clearfix bqQt"}) #find the html loops to scrape

In [282]:
len(debug_soup_quote[5])

7

In [290]:
re.search('[\.\!\?\']\w.*$',debug_soup_quote[5].get_text().replace('\n','').replace('\'','')).group(0)

'.George Burns'

7

In [247]:
debug_soup_quote[5].get_text().replace('\n','')

" You can't help getting older, but you don't have to get old.George Burns"

In [244]:
re.search('[\.\!\?\']\w.*$',debug_soup_quote[5].get_text().replace('\n','').replace('\'','')).group(0)

'.George Burns'

In [248]:
len(debug_soup_quote[4])

5

In [295]:
re.search('[\.\!\?\']\w.*$',debug_soup_quote[9].get_text().replace('\n','').replace('\'','')).group(0)

'.Arthur Schopenhauer'

In [217]:
debug_soup_quote[4].contents[3].get_text()

'Victor Hugo'

In [196]:
debug_soup_quote[4].contents[3].get_text()

'Victor Hugo'